# LASER + Logistic Regresion (Language Family)

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import requests
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from sklearn.metrics import accuracy_score, f1_score,confusion_matrix

## Utility Functions

In [117]:
# Ensure that you have the API running before running this.
def generate_LASER_embedding(sequence, lang='en'):
    if not sequence or len(sequence) < 0:
        print('Please pass in an input sequence to generate an embedding.')
    url = "http://128.61.46.191:80/vectorize"
    params = {"q": sequence, "lang": lang}
    resp = requests.get(url=url, params=params).json()
    arr = np.array(resp["embedding"])
    return arr

In [122]:
def generate_embeddings_for_dataset(ifname, lang):
    '''
    Generates embeddings and saves it into a CSV given an input file path and a language.
    '''
    print(f'Generating input embeddings for {ifname}')
    embeddings = []
    df = pd.read_csv(f'{DATA_DIR}/{ifname}')
    print(f'Total embeddings to generate: {df.shape[0]}')
    for idx, row in df.iterrows():
        if (idx+1) % (df.shape[0] // 25) == 0:
            print(f'Completed {idx+1} embeddings')
        embedding = generate_LASER_embedding(row['text'], lang=lang)[0]
        embedding = np.append(embedding, [int(row['hs'])])
        embeddings.append(embedding)
    embeddings = np.array(embeddings)
    # Save each entry of the embedding array as a column in the dataframe.
    columns = [str(i) for i in range(1024)]
    columns.append('hs')
    embedding = pd.DataFrame(embeddings, columns=columns)
    embedding.hs = embedding.hs.astype(int)
    ofname = ifname.split('.')[0] + '_embeddings.csv'
    embedding.to_csv(f'{DATA_DIR}/{ofname}', index=False)
    print(f'Wrote embedding dataframe to {DATA_DIR}/{ofname}')

In [5]:
def train_and_test_LR_model(ifnames, c=0.01):
    dfs = []
    for ifname in ifnames:
        df = pd.read_csv(f'{DATA_DIR}/{ifname}')
        dfs.append(df)
    df = pd.concat([df])
    X_cols = [str(i) for i in range(1024)]
    X, y = df[X_cols], df['hs']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
    print(f'Training examples: {X_train.shape[0]}')
    model = LogisticRegression(C=c, solver='lbfgs', class_weight='balanced', random_state=42)
    model.fit(X_train, y_train)
    print('Training complete!')
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Accuracy: {accuracy}')
    f_score = f1_score(y_test, y_pred ,average='macro')
    print(f'Macro F1: {f_score}')

In [6]:
DATA_DIR = '../data/all-processed'